## Gender Classification

### Model - 

### Input - Images

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('custom_dataset.csv')
dataset.head()

,id,sex,weight,height,bmi
0,A00147,Male,83.91,1.70,28.97
1,A00220,Male,70.31,1.85,20.45
2,A00360,Male,75.75,1.75,24.66
3,A00367,Male,111.13,1.83,33.23
4,A01054,Male,75.30,1.70,26.00


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61110 entries, 0 to 61109
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      61110 non-null  object 
 1   sex     61109 non-null  object 
 2   weight  60716 non-null  float64
 3   height  60728 non-null  float64
 4   bmi     60715 non-null  float64
dtypes: float64(3), object(2)
memory usage: 2.3+ MB
